In [2]:
import json
import pandas as pd
import firebase_admin
from datetime import datetime, timedelta
from firebase_admin import credentials
from firebase_admin import firestore

In [17]:

df = pd.read_csv("../Text Normalization/Tweets.csv")
df = df[0:500]

df.head()

,Unnamed: 0,Tweet,Location,year,month,day,Sentiment
0,0,اللي بينا مايتقدر عمرودياب فودافون,"Cairo, Egypt",2022,4,4,negative
1,1,ما بعد فوق قمة الاستهتار والاهمال واللا مبالاة...,Egypt,2022,4,4,negative
2,2,عاجل فضيحة فودافون تتسبب في أزمة بعد تسريببيان...,dokki,2022,4,4,negative
3,3,وحدة فودافون,مصر,2022,4,3,neutral
4,4,فودافون وحده وحده وحده وحده ...,مصر,2022,4,3,positive


In [18]:
#df.dropna(inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  500 non-null    int64 
 1   Tweet       500 non-null    object
 2   Location    500 non-null    object
 3   year        500 non-null    int64 
 4   month       500 non-null    int64 
 5   day         500 non-null    int64 
 6   Sentiment   500 non-null    object
dtypes: int64(4), object(3)
memory usage: 27.5+ KB


In [4]:
CredNew = credentials.Certificate('./smartcc-960f7-firebase-adminsdk-ijo0i-27aa89d85e.json')
new = firebase_admin.initialize_app(CredNew,name="new")

db = firestore.client(new)

In [5]:
print(df.shape[0])

500


In [25]:
def getPastDateAndTime(daysBefore):
    currentDate = str(datetime.now() - timedelta(days = daysBefore)) 
    splitDateList = currentDate.split(" ")
    dateOnly = splitDateList[0]
    timeOnly = splitDateList[1].split(".")[0]
    dateWithTime = dateOnly + " " + timeOnly

    return [dateOnly, dateWithTime]



def uploadDataToFirebase (collectionName, searchKeyword, df, dataLimit):
    dfLength = df.shape[0]
    iterationCount = 1
    minLimit = 0
    maxLimit = None
    
    if (dfLength > dataLimit):
        iterationCount = int(dfLength / dataLimit) + 1

    for i in range(0, iterationCount):
        firebaseData = {}
        
        if (iterationCount == 1):
            firebaseData = json.loads(json.dumps(df[0:].to_dict()))
        else:
            if (i == (iterationCount - 1)):
                maxLimit = None
            else:
                maxLimit = dataLimit * (i + 1)
                
            minLimit = dataLimit * i 

            firebaseData = json.loads(json.dumps(df[minLimit:maxLimit].to_dict()))
            
        currentDate, currentDateAndTime = getPastDateAndTime(1)
        
        db.collection(collectionName).document(searchKeyword).collection(currentDate).document(currentDateAndTime).set(firebaseData)

    

In [26]:
uploadDataToFirebase("test4", "فودافون", df, 2500)

1
